There are the libraries needed to make some web scraping.

In [1]:
import requests
import pickle
from tqdm import tqdm
from bs4 import BeautifulSoup
import time
import re
import time
import warnings
warnings.filterwarnings("ignore")

There are about $1700$ *immobiliare* pages for announcements in Rome with $25$ links per page. Actually we work on only $700$ pages since we need $10K$ documents.

In [ ]:
%%time
records = []
count = 1
start_time = time.time()

# 670 pages are enough to get 10K announcements
for i in tqdm(range(1, 671)):
    r = requests.get('https://www.immobiliare.it/vendita-case/roma/?criterio=rilevanza&pag=' + str(i), verify = False)  
    soup = BeautifulSoup(r.text, 'html.parser')

    for block in soup.find_all('div', class_ = 'listing-item_body'):
        try:
            list_ = block.find('ul', class_ = 'listing-features list-piped')
            title = 'announcement_' + str(count)
            prezzo_ = block.findAll('li', class_ = 'lif__item lif__pricing')
            prezzo = prezzo_[-1].text.strip()
            locali_ = block.findAll('span', class_ = 'text-bold')
            locali = locali_[-3].text
            superfici = locali_[-2].text
            bagni = locali_[-1].text
            piano = block.div.abbr['title']
            
            # we replace this strings with the number 0
            if (piano == 'T' or piano == 'Piano terra'): 
                piano = 0
                
            # we want to work only with numeric prices
            elif (piano.isdigit() == False or prezzo == 'EZZO SU RICHIESTA'):
                continue
            # there are some discount percentages, so we take the first number which is the discounted one
            if ('%' in prezzo):
                prezzo = '€' + prezzo.split('€')[1]
                
            url = block.find('a', href = True)

            if ('https://' in url['href']):
                ann = requests.get(url['href'])
                
            # there are some 'broken' links (starting with 'http')
            else:
                ann = requests.get('https://www.immobiliare.it' + url['href'])
                
            
            data = ann.text
            soup1 = BeautifulSoup(data, 'html.parser')

            # we want to retrieve the entire description
            desc = soup1.find('div', attrs = {'role':'contentinfo'}).text.strip()
            
            # this is the list with all the infos
            records.append((title, prezzo, locali, superfici, bagni, piano, desc))
            count += 1
                        
        except:
            pass
            

    time.sleep(1)
    
print(time.time() - start_time)

In order to keep the announcements, we save the **records** list as a *.txt* file.

In [ ]:
with open("records_list.txt", "wb") as fp:  
    pickle.dump(records, fp)